# APIs


## Tasks Today:
1) <b>Requests and the pokemon API </b> <br>
 

# working with APis

<p> What exactly is an API? <br> <br>
API is the acronym for Application Programming Interface, which is a software intermediary that allows two applications to talk to each other. Each time you use an app like Facebook, send an instant message, or check the weather on your phone, you're using an API. </p>

### The Poke API  allows you to retreive a pokemon's information from PokeAPI https://pokeapi.co/



In [13]:
# making an API call
import requests

api_endpoint = "https://pokeapi.co/api/v2/pokemon/"

poke_name = input('What Pokemon are you looking for? ')
response = requests.get(f"{api_endpoint}{poke_name}")

if response.status_code == 200:
    data = response.json()
    
    print(f"Success. 'data' for {poke_name} ready")
else:
    print(f"Error: Status Code {response.status_code}")

What Pokemon are you looking for? psyduck
Success. 'data' for psyduck ready


### Display a Pokemon's name, weight, abilities, and types

In [14]:
# get the name
name = data['name']

'psyduck'

In [15]:
# get types
type = data['types'][0]

{'slot': 1,
 'type': {'name': 'water', 'url': 'https://pokeapi.co/api/v2/type/11/'}}

In [16]:
# get weight
weight = data['weight']

196

In [19]:
# get abilities
abilities = data['abilities']

In [20]:
[ability_dict['ability']['name'] for ability_dict in abilities]

['damp', 'cloud-nine', 'swift-swim']

In [ ]:
# Create a structure for a single pokemon

In [27]:
pokemon_dict = {
    'name': data['name'],
    'type': [type_dict for type_dict in data['types'][0]]
}

#### Create a function to Pull in your own Pokemon's data 

In [33]:
def poke_api_call(poke_name):
    response = requests.get(f"https://pokeapi.co/api/v2/pokemon/{poke_name}")
    if response.status_code == 200:
        data = response.json()
        print(f"Success. 'data' for {poke_name} ready")
        pokemon_dict = {key: data[key] for key in ['name', 'types', 'weight', 'abilities']}
        return pokemon_dict
    else:
        print(f"Error: Status Code {response.status_code}")
        
poke_api_call('jirachi')

Success. 'data' for jirachi ready


{'name': 'jirachi',
 'types': [{'slot': 1,
   'type': {'name': 'steel', 'url': 'https://pokeapi.co/api/v2/type/9/'}},
  {'slot': 2,
   'type': {'name': 'psychic', 'url': 'https://pokeapi.co/api/v2/type/14/'}}],
 'weight': 11,
 'abilities': [{'ability': {'name': 'serene-grace',
    'url': 'https://pokeapi.co/api/v2/ability/32/'},
   'is_hidden': False,
   'slot': 1}]}

Choose your pokemon

In [ ]:
from random import randint
# Random number generated for each pokemon id
random_team = [randint(1,898) for i in range(6)]

your_team = ['electabuzz', 'haunter','tyranitar','blaziken','marowak','dragonair']


#### Use your function to create a dictionary of your Johto League 6  (favorite 6 pokemon)

In [ ]:
# Place all 6 of your pokemon on the object below, each pokemon should have at least as much info as Pikachu did.
my_six_pokemon = {}


## Lets create a class called 'Pokemon' and create our pokemon as instances

In [ ]:
class Pokemon():
    pass

### Let's Catch some Pokemon

In [ ]:
pokedex = {}


## Exercise 1:

### Create a Method prints an image of your pokemon

<p>HINT: You may need another attribute as well to store your image url within. </p>

In [ ]:
# Display an image in Jupyter notebook
from IPython.display import Image

# display(Image( 'https://i.redd.it/45n4mhusa8l41.jpg', width = 300))


In [ ]:
# recreate your pokemon class here
class Pokemon:
    pass

In [ ]:
# Calling our new method


## Exercise 2:

### Create a Method that evolves your Pokemon
If your pokemon can't evolve any further print a message that says "\<name of pokemon> can't evolve."

Now let's evolve a few

#  Final Exercise: <br> <br>Create a Move_Tutor Class that will allow the Pokemon Class to inherit a move list.
<br>
<p>for an added bonus you can make sure that if a pokemon has 4 moves the user can choose one of them to replace with a new move. </p>

In [ ]:
class Move_Tutor:
    def __init__(self):
        self.move_list = []

In [ ]:
pikachu.teach_move()


In [ ]:
pikachu.show_moves()